In [19]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
import plotly.graph_objects as go
from dustmaps.edenhofer2023 import Edenhofer2023Query

plot_save_directory = "./savedplots/"

In [20]:
def cylinder(r, h, nt=100, nv=50):
    """
    Parametrize cylinder of radius r, height h, at startpoint z=a
    """
    theta = np.linspace(0, 2*np.pi, nt)
    v = np.linspace(-h/2, h/2, nv)
    theta, v = np.meshgrid(theta, v)

    y = r*np.cos(theta)
    z = r*np.sin(theta)
    x = v
    return x, y, z

def filledcylinder(r, h, nt=100, nh=50, nr=50):
    """
    Parametrize cylinder of radius r, height h, at startpoint z=a
    """
    theta = np.linspace(0, 2*np.pi, nt)
    v = np.linspace(-h/2, h/2, nh)
    rr = np.linspace(0, r, nr)
    rr, theta, v = np.meshgrid(rr, theta, v)

    y = rr*np.cos(theta)
    z = rr*np.sin(theta)
    x = v
    return x, y, z

def rotationMatrix(a=0, b=0, c=0):
    """
    Return the Rotation Matrix with yaw a about z, pitch b about y, and roll c about x
    """
    Ra = np.array([
        [np.cos(a), -np.sin(a), 0],
        [np.sin(a), np.cos(a), 0],
        [0, 0, 1]])

    Rb = np.array([
        [np.cos(-b), 0, np.sin(-b)],
        [0, 1, 0],
        [-np.sin(-b), 0, np.cos(-b)]
    ])

    Rc = np.array([
        [1, 0, 0],
        [0, np.cos(c), -np.sin(c)],
        [0, np.sin(c), np.cos(c)]
    ])

    return Ra@Rb@Rc

_____

# Define Cylinder, Get Data

In [118]:
#define the cylinder
R = 40
H = 200

nt = 90
nh = 105
nr = 100
x1, y1, z1 = filledcylinder(r=R, h=H, nt=nt, nh=nh, nr=nr)
#Get rotation Matrix
RotM = rotationMatrix(a=-(1.7/18)*np.pi, b=(0.5/18)*np.pi, c=0)
#Define Translation Vector
t=np.array((80, 190, 40))

v_f_min = 1e-6
v_f_max = 8e-4
v_min = 2e-4
v_max = 5e-3

In [119]:
#Stack coordinates for rotation
S = np.stack((x1, y1, z1), axis=2)

In [120]:
#Rotate by multiplying with rotMatrix
rotatedStack = RotM@S

#Separate again, and translate
xx = rotatedStack[:,:,0,:]+t[0]
yy = rotatedStack[:,:,1,:]+t[1]
zz = rotatedStack[:,:,2,:]+t[2]

In [121]:
#Visualize what the shell of the cylinder looks like:
cyl = go.Surface(x = xx[:,-1,:], y = yy[:,-1,:], z = zz[:,-1,:],
    opacity = 0.3,
    colorscale = [[0, "green"], [1, "green"]],
    showscale = False,
    name = "surface -1",
    showlegend = True
)

cyl2 = go.Surface(x = x1[:,-1,:]+t[0], y = y1[:,-1,:]+t[1], z = z1[:,-1,:]+t[2],
    opacity = 0.1,
    colorscale = [[0, "red"], [1, "red"]],
    showscale = False,
    name = "surface -1",
    showlegend = True
)

#cyl_2 = go.Surface(x = xx_2[:,-1,:], y = yy_2[:,-1,:], z = zz_2[:,-1,:],
#    opacity = 0.3,
#    colorscale = [[0, "orange"], [1, "orange"]],
#    showscale = False,
#    name = "a second cylinder",
#    showlegend = True
#)

center = go.Scatter3d(
    x=[t[0]],
    y=[t[1]],
    z=[t[2]],
    mode = "markers",
    marker = dict(
        size=3, 
        color="yellow"
    ),
    name="center")



layout = go.Layout(template="plotly_white", showlegend=True, title="Test", height=500, width=800, scene = dict(
    aspectmode = 'manual',
    aspectratio = dict(x = 1, y = 1, z = 1)))

fig = go.Figure(data=(center, cyl, cyl2), layout=layout)
fig.write_html(plot_save_directory + "Cylindertest.html")
fig.show()

In [122]:
# Define skycoords using the coordinates of the points in the cylinder:
sc_cylinder = SkyCoord(
    xx.flatten() * u.pc,
    yy.flatten() * u.pc,
    zz.flatten() * u.pc,
    frame = 'galactic',
    representation_type = 'cartesian'
)

sc_cylinder.representation_type ="spherical"

#Definde SkyCoords in a orthogonal grid:
#xt = np.linspace(np.min(xx), np.max(xx), 20)
#yt = np.linspace(np.min(yy), np.max(yy), 20)
#zt = np.linspace(np.min(zz), np.max(zz), 20)

xt = np.arange(-20, 220, 10)
yt = np.arange(100, 280, 10)
zt = np.arange(-30, 110, 10)

xt_grid, yt_grid, zt_grid = np.meshgrid(xt, yt, zt)

sc_grid = SkyCoord(
    xt_grid.flatten() * u.pc,
    yt_grid.flatten() * u.pc,
    zt_grid.flatten() * u.pc,
    frame = 'galactic',
    representation_type = 'cartesian'
)

sc_grid.representation_type = "spherical"

In [123]:
#Query Edenhofer2023:
dust_cyl = Edenhofer2023Query().query(sc_cylinder)
#dust_grid = Edenhofer2023Query().query(sc_grid)

Optimizing map for querying (this might take a couple of seconds)...


In [115]:
#Definde the Volumes:
volume_grid_flat = go.Volume(
    x = xt_grid.flatten(),
    y = yt_grid.flatten(),
    z = zt_grid.flatten(),
    value = dust_grid,
    flatshading = True,
    opacityscale = [[0,0], [1, 1]],
    isomin = v_f_min,
    isomax = v_f_max,
    surface = dict(show=True, count=1)
)

volume_grid = go.Volume(
    x = xt_grid.flatten(),
    y = yt_grid.flatten(),
    z = zt_grid.flatten(),
    value = dust_grid,
    opacity = 0.3,
    isomin = 2e-4,
    isomax = 5e-3,
    surface_count = 10
)

In [117]:
x_range = (np.min((np.min(xx), np.min(xt))), np.max((np.max(xx), np.max(xt))))
y_range = (np.min((np.min(yy), np.min(yt))), np.max((np.max(yy), np.max(yt))))
z_range = (np.min((np.min(zz), np.min(zt))), np.max((np.max(zz), np.max(zt))))

yscale = (np.max(y_range)-np.min(y_range))/(np.max(z_range)-np.min(z_range))
xscale = (np.max(x_range)-np.min(x_range))/(np.max(z_range)-np.min(z_range))

#Visualize the result
layout_3d = go.Layout(
    template = "plotly_white",
    #paper_bgcolor = "black",
    #plot_bgcolor = "black",
    title = "Dust",
    scene = dict(
        aspectmode = 'manual',
        aspectratio = dict(x = xscale, y = yscale, z = 1),
        xaxis = dict(title = "x", range = x_range),
        yaxis = dict(title = "y", range = y_range),
        zaxis = dict(title = "z", range = z_range),
    ),
    height=500,
    width=800
)

fig_3d_dust = go.Figure(data = (volume_grid_flat, cyl, center), layout = layout_3d)
fig_3d_dust.write_html(plot_save_directory + "Cylindertest.html")
fig_3d_dust.show()

In [100]:
print("xrange: " + str(x_range))
print("yrange: " + str(y_range))
print("zrange: " + str(z_range))

xrange: (-13.173771147526978, 213.18358028259408)
yrange: (99.34147806253777, 280.701009969353)
zrange: (-29.989097740589912, 109.9890977405899)


_____

# Calc Mass

In [127]:
#selektiere nach xx[Winkel, radius/schale, Höhe]
xx[:,0,2]

array([-287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708427,
       -287.27708427, -287.27708427, -287.27708427, -287.27708

In [124]:
#Reshape dust_cyl, so i can index it the same way
dust_cyl_mat = np.reshape(dust_cyl, (nt, nr, nh))

In [125]:
#Calculate Volume of the cylindrical elements in pc^3
def dv_i(i):
    return (np.pi*(2*i+1)*R**2*H)/(nr**2*nt*nh)

In [126]:
#Schale 0
Schale_0 = np.sum(dust_cyl_mat[:,0,:]*dv_i(0), where=dust_cyl_mat[:,0,:]>=1.6e-3)

In [127]:
#Sum over all layers, only where density>=1.6e-3
layers = np.zeros(shape=(nr,))
for i in np.arange(nr):
    layers[i] = np.sum(dust_cyl_mat[:,i,:]*dv_i(i), where=dust_cyl_mat[:,i,:]>=(v_f_max+v_f_min)/2)

TotalDensity = np.sum(layers)

In [128]:
TotalMass = 1.37*1.672621911e-27*4.857e58*TotalDensity
TotalMass_sol = TotalMass/1.988e30

In [129]:
print(TotalMass_sol)

19322.022750432563


_____

In [ ]:
#Define x2, y2, z2 as points in cylinder
R_2 = 75
H_2 = 275

#nt = 18
#nh = 22
#nr = 15
x2, y2, z2 = filledcylinder(r=R_2, h=H_2, nt=nt, nh=nh, nr=nr)
#Get rotation Matrix
RotM_2 = rotationMatrix(a=(7/18)*np.pi, b=-(0.5/18)*np.pi, c=0)
#Define Translation Vector
t_2=(-490, -470, 370)

In [ ]:
S_2 = np.stack((x2, y2, z2), axis=2)

In [ ]:
rotatedStack_2 = RotM_2@S_2

xx_2 = rotatedStack_2[:,:,0,:]+t_2[0]
yy_2 = rotatedStack_2[:,:,1,:]+t_2[1]
zz_2 = rotatedStack_2[:,:,2,:]+t_2[2]